In [ ]:
import csv
import pandas as pd
from transformers import pipeline
# Open and read the CSV file using the csv module
csv_file_path = "Nasdaq_stocks.csv"
rows = []

with open(csv_file_path, 'r', encoding='utf-8', errors='ignore') as csvfile:
    csvreader = csv.reader(csvfile)
    for row in csvreader:
        rows.append(row)

# Create a DataFrame from the list of rows
stocks = pd.DataFrame(rows[1:], columns=rows[0])

stocks.shape

FileNotFoundError: [Errno 2] No such file or directory: 'Nasdaq_stocks.csv'

In [ ]:
stocks_data = stocks[['Date','Article_title','Stock_symbol']]
# Convert 'date' column to datetime type
stocks_data.loc[:, 'Date'] = pd.to_datetime(stocks_data['Date'], utc=True)

# Filter data for the years 2015 to 2023
start_date = pd.Timestamp('2019-01-01',tz="UTC")
end_date = pd.Timestamp('2023-12-31',tz="UTC")

stocks_data = stocks_data[(stocks_data['Date'] >= start_date) & (stocks_data['Date'] <= end_date)]
stocks_data.loc[:, 'Date'] = stocks_data['Date'].astype('str')
# Extract date component as string 'YYYY-MM-DD' using string manipulation
stocks_data.loc[:, 'Date'] = stocks_data['Date'].str.split().str[0]

In [ ]:
stocks_data = stocks_data.rename(columns={'Article_title': 'headlines'})
stocks_data["Stock_symbol"].value_counts()


Stock_symbol
GOOG    9883
NVDA    9533
AAPL    9338
INTC    9007
MSFT    8737
Name: count, dtype: int64

In [ ]:
from multiprocessing import Pool, cpu_count
df = stocks_data

In [ ]:
#!pip install transformers
# Initialize sentiment analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

# Function to process a batch of headlines
def process_batch(batch):
    return [sentiment_pipeline(text)[0]['label'] for text in batch]

# Split the dataframe into batches
batch_size = 100  # Adjust the batch size as needed
batches = [df['headlines'][i:i + batch_size] for i in range(0, len(df), batch_size)]

# Use multiprocessing to process the batches
with Pool(cpu_count()) as pool:
    results = pool.map(process_batch, batches)

# Flatten the list of results
df['sentiment'] = [item for sublist in results for item in sublist]

# Convert sentiment labels to numerical scores
sentiment_to_score = {
    'POSITIVE': 1,
    'NEGATIVE': -1,
    'NEUTRAL': 0
}
df['sentiment_score'] = df['sentiment'].map(sentiment_to_score)


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [ ]:
# Calculate average sentiment score for each day
daily_sentiment = df.groupby(['Date','Stock_symbol']).agg({
    'sentiment_score': 'mean'
}).reset_index()

file_path = 'stocks_data_with_sentiment.csv'
daily_sentiment.to_csv(file_path, index=False)

In [ ]:
df["sentiment"].value_counts()

NameError: name 'df' is not defined

In [ ]:
import pandas as pd
df = pd.read_csv("stocks_data_sentiment_score.csv")

In [ ]:
df.head(5)

,Date,Stock_symbol,sentiment_score
0,2019-01-01,INTC,-1.000000
1,2019-01-02,GOOG,-1.000000
2,2019-01-02,INTC,-0.250000
3,2019-01-02,NVDA,-1.000000
4,2019-01-03,GOOG,-0.333333


In [ ]:
# Group by 'Stock_symbol' and count rows based on sentiment_score conditions
count_positive = df[df['sentiment_score'] > 0].groupby('Stock_symbol').size()
count_negative = df[df['sentiment_score'] < 0].groupby('Stock_symbol').size()
count_neutral = df[df['sentiment_score'] == 0].groupby('Stock_symbol').size()

# Fill missing values with 0
count_positive = count_positive.fillna(0)
count_negative = count_negative.fillna(0)
count_neutral = count_neutral.fillna(0)

print("Count of rows where sentiment score is > 0:")
print(count_positive)

print("\nCount of rows where sentiment score is < 0:")
print(count_negative)

print("\nCount of rows where sentiment score is = 0:")
print(count_neutral)


Count of rows where sentiment score is > 0:
Stock_symbol
AAPL    100
GOOG    286
INTC    333
MSFT     97
NVDA    249
dtype: int64

Count of rows where sentiment score is < 0:
Stock_symbol
AAPL     494
GOOG    1185
INTC    1061
MSFT     451
NVDA     700
dtype: int64

Count of rows where sentiment score is = 0:
Stock_symbol
AAPL     45
GOOG    245
INTC    188
MSFT     46
NVDA    151
dtype: int64
